# Plan

1. complete functions
2. change variables in functions that are found in 'arg =' file
3. improve readablilty (add spaces between maths)
4. understand ode from matlab into python (see scipy from ph456 and online)

# Imports

In [66]:
import numpy as np
import scipy.integrate
from math import sqrt, pi, log
from numpy import linalg as LA



# Environment Set Up

In [53]:

Delta = 0.2*2*pi #Delta is sheath width/thickness
Rb    = 0.28*2*pi #rb is blowout radius
dRb   = 0 # dRb is derivative of rb

xi_0   = 0 # initial conditions for RHS
xi_max = -2.25*Rb #max intrgration limits
dxi    = -0.01 # integration steps

xi_inj  = 0 #found in load_arg 
xi_tr   = 0 #found in load_arg / what is this?

n     = 0 # found in load_arg / what is this?
rc    = 0 #0.5*2*pi
rB    = rc/2 # half of rc

E_inj = 0   
rhoC  = 0 
dE    = 0 

gammaB = 10000
epsB   = 1/gammaB**2
vB     = sqrt(1-epsB)

lamb_fac = 0

In [54]:
arg = {'Delta': Delta, 'n' : n, 'rc' : rc, 'rhoC' : rhoC, 'Rb' : Rb, 'E_inj' : E_inj, 
       'dE': dE, 'xi_tr' : xi_tr, 'xi_inj' : xi_inj, 'lamb_fac' : lamb_fac }
#arg['rhoC']
#stores arguments in dictionary

In [55]:
arg_part = {'Delta': Delta, 'n' : n, 'rc' : rc, 'rhoC' : rhoC, 'Rb' : Rb, 'E_inj' : E_inj, 
       'dE': dE, 'xi_tr' : xi_tr, 'xi_inj' : xi_inj, 'dt' : 0, 'En' : 0, 'L' : 0, 'vB' : 0}
#stores arguments in dictionary

# Key Functions

In [56]:
def int_rho(r,arg):
    if n == 0:
        INT_RHO = rhoC * rc**2 / 2 + 1 * ((np.power(r, 2)) - rc**2) / 2  # what is rhoC ?
    else:                                                                #check this equation's indexes
        INT_RHO = rhoC * (np.power(rc, 2)) / 2 + (rhoC * Rb**n - rc**n) / \
        (Rb**n - rc**n) * (np.power(r,2)) / 2 + (1-rhoC)/(Rb**n - rc**n) * (np.power(r,(n+2)))/(n+2) - \
        (rhoC*Rb**n - rc**n) / (Rb**n - rc**n) * rc**2 / 2 + (1-rhoC) / (Rb**n - rc**n) *rc**(n+2)/(n+2)

# conditional assignment INT_RHO(r<rc) = rhoC*r(r<rc).^2/2;
#return INT_RHO



In [57]:
def rho(r,arg):
    
    if n == 0:
        RHO = np.ones(r)
    else:
        RHO = (Rb**n - (np.power(r, n))) * rhoC / (Rb**n - rc**n) + ((np.power(r, n)) - rc**n) / (Rb**n - rc**n)
        
#end func
#conditional assignment
#RHO(r<rc) = rhoC; for every element r<rc set equal to rhoC?

In [58]:
def give_arg(): # maybe don't need this or give_arg_part as dictionary is already created
    arg['n']      = n # order of transverse plasma profile
    arg['rc']     = rc # 
    arg['rhoC']   = rhoC #
    arg['Rb']     = Rb 
    arg['E_inj']  = E_inj
    arg['dE']     = dE
    arg['xi_tr']  = xi_tr
    arg['xi_inj'] = xi_inj
    arg['Delta']  = Delta #sheath width
    return arg

In [59]:
def drho(r):
    #drho~nr^(n-1) | drho(r<rc)=0

    if arg['n'] == 0:
        DRHO = np.zeros(r.shape)
        
    elif:        
   #     DRHO = (0 - n * np.power(r, n-1)) * arg['rhoC'] / (arg['Rb']**n - rc**n) + (n * np.power(r, n-1) - 0) / (Rb**n - rc**n)    
    
    # conditional assignment here
    else r<rc:
        DHR0=....
    
    return DRHO


In [60]:
def RHS(xi,R0,arg):
    drb  = R0[0]
    rb   = R0[1]
    dsi = drho(rb,arg)
    si = rho(rb,arg)
    SI = int_rho(rb,arg)

    eps = arg['Delta']/rb
    X = 1 +eps
    dX = -(X-1)**2/Delta
    ddX = -2*(X-1)*dX/Delta

    delta = X**2/(X**2-1)
    ddelta = -2*X*dX/(X**2-1)**2
    dddelta = -2 * dX**2 / (X**2-1)**2 -2*X*ddX/(X**2-1)**2 +8*X**2*dX**2/(X**2-1)**3

    beta = 2*delta*log(X) -1
    dbeta = 2*ddelta*log(X) +2*delta*dX/X
    ddbeta = 2*dddelta*log(X) +4*ddelta*dX/X -2*delta*dX**2/X**2 +2*delta*ddX/X

    A = (1 +SI/2)*rb +(si*rb**2/4 +SI/2)*rb*beta +SI*rb**2/4*dbeta
    B = si*rb**2/2 +(3*si*rb +dsi*rb**2)*rb*beta/4 +(SI +si*rb**2)*rb*dbeta/2 +SI*rb**2*ddbeta/4
    C = SI/2*(1 +1/(1 +SI*beta/2)**2)


#% A = SI*rb/2
#% B = si*rb**2/2
#% C = SI/2

    if lamb_fac == 1 and xi <= xi_inj and xi >= xi_tr:
        Lambda = SI/2 +E_inj**2*rb**2/(2*SI)
    elif lamb_fac == 2 and xi <= xi_inj and xi >= xi_tr:
        Lambda = SI/2 +(dE*xi -dE*xi_inj -E_inj)**2*rb**2/(2*SI) +dE*(dE*xi -dE*xi_inj -E_inj)*rb**3/(2*sqrt(2*F(rb,Rb,arg)))
    elif lamb_fac == 3 and xi >= xi_inj and xi <= xi_tr:    
        Lambda = SI/2 +(dE*xi -dE*xi_inj +E_inj)**2*rb**2/(2*SI) -dE*(dE*xi -dE*xi_inj +E_inj)*rb**3/(2*sqrt(2*F(rb,Rb,arg)))
    else:
        Lambda = 0

        dR0    = np.zeros(2,1,dtype = int)
        dR0[1] = drb
        dR0[0] = ( Lambda -B*drb**2 -C ) /A
        

# Main Codeflow

In [61]:
def test():
    arg['Delta'] = 7

In [62]:
arg['Delta']

1.2566370614359172

In [64]:
test()

In [71]:
test = [dRb, Rb]
[XI0,R] = scipy.integrate.solve_ivp(RHS(xi,rb,arg), [xi_0,dxi,xi_max], test, options)

NameError: name 'xi' is not defined